**Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

base_dir = "/content/drive/MyDrive/PA/data/preprocessed"

from google.colab import drive
drive.mount('/content/drive')

!nvidia-smi

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fri Dec 26 18:49:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off

**Fitted parameters vs pressure**

In [ ]:
# ==== Parameters & Errors vs Pressure (from MERGED NPZ with labels) ====
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntRangeSlider, Dropdown, Checkbox
import os

FIT_DIR = "/content/drive/MyDrive/data/fit"

# Datasets you want
DATASETS = ["1_lowering", "2_lowering", "1_increasing"]

# define which labels are considered "good"
GOOD_LABELS = {"Jetting"}

# --- selector list: 6 params + 3 metrics ---
def _u_tau(x, micro):  # seconds -> microseconds option
    return (x*1e6, "τ (µs)") if micro else (x, "τ (s)")

def _u_f(x, kilo):     # Hz -> kHz option
    return (x/1e3, "f (kHz)") if kilo else (x, "f (Hz)")

selector = {
    "A":      ("A (Amplitude)",        "params6", 0, None,          None),
    "tau":    ("τ (Damping time)",     "params6", 1, "micro",       _u_tau),
    "f":      ("f (Frequency)",        "params6", 2, "kilo",        _u_f),
    "phi":    ("φ (Phase)",            "params6", 3, None,          None),
    "C":      ("C (Baseline offset)",  "params6", 4, None,          None),
    "D":      ("D (Baseline slope)",   "params6", 5, None,          None),
    "rmse":   ("RMSE",                 "rmse",    None, None,       None),
    "nrmse":  ("NRMSE",                "nrmse",   None, None,       None),
    "Q":      ("Q factor (π f τ)",     "Q",       None, None,       None),
}

def load_merged_for_dataset(dataset_key: str):
    # parse "2_lowering" -> replicate=2, version="lowering"
    rep = int(dataset_key.split("_")[0])
    ver = dataset_key.split("_", 1)[1]

    merged_fn = f"merged_rep_{rep}_{ver}_bankcorrected_params6_Q_rmse_nrmse_labels.npz"
    merged_fp = os.path.join(FIT_DIR, merged_fn)
    if not os.path.exists(merged_fp):
        raise FileNotFoundError(f"Merged file not found:\n{merged_fp}")

    merged     = np.load(merged_fp, allow_pickle=True)
    params6    = merged["params6_all"]
    rmse_all   = merged["rmse_all"]
    nrmse_all  = merged["nrmse_all"]
    Q_all      = merged["Q_all"]
    pressures  = merged["pressures"].astype(float)
    nozzles    = merged["nozzles"]
    labels_raw = merged["labels"]

    labels_str = np.char.strip(labels_raw.astype(str))
    label_set = {lbl for lbl in labels_str.ravel() if lbl not in ("", "nan", "None")}
    label_list = sorted(label_set)

    n_nozzles, n_pressures = rmse_all.shape

    print(f"\n Loaded merged arrays from: {merged_fp}")
    print(f"params6: {params6.shape}  rmse: {rmse_all.shape}  nrmse: {nrmse_all.shape}  Q: {Q_all.shape}")
    print(f"labels:  {labels_raw.shape}")
    print("Available labels:", ", ".join(label_list) if label_list else "(none)")

    return dict(
        merged_fp=merged_fp,
        params6=params6,
        rmse_all=rmse_all,
        nrmse_all=nrmse_all,
        Q_all=Q_all,
        pressures=pressures,
        nozzles=nozzles,
        labels_str=labels_str,
        label_list=label_list,
        n_nozzles=n_nozzles,
        n_pressures=n_pressures,
    )

# keep current dataset in a small state dict
STATE = {}

def plot_param(dataset="1_lowering",
               which="A",
               nozzle_range=(0, 9),
               use_kHz=True,
               use_us=True,
               label_filter="(all)"):

    # (re)load if dataset changed or first run
    if (not STATE) or (STATE.get("dataset") != dataset):
        STATE.clear()
        STATE.update(load_merged_for_dataset(dataset))
        STATE["dataset"] = dataset

    params6     = STATE["params6"]
    rmse_all    = STATE["rmse_all"]
    nrmse_all   = STATE["nrmse_all"]
    Q_all       = STATE["Q_all"]
    pressures   = STATE["pressures"]
    nozzles     = STATE["nozzles"]
    labels_str  = STATE["labels_str"]
    n_nozzles   = STATE["n_nozzles"]

    name, src, idx, default_units, unit_fn = selector[which]
    start, end = nozzle_range
    start = int(np.clip(start, 0, n_nozzles-1))
    end   = int(np.clip(end,   0, n_nozzles-1))
    if end < start:
        start, end = end, start

    # pick source matrix
    if src == "params6":
        mat = params6[:, :, idx]
        ylab = name
        if which == "tau" and unit_fn is not None:
            mat, ylab = unit_fn(mat, use_us)
        elif which == "f" and unit_fn is not None:
            mat, ylab = unit_fn(mat, use_kHz)
    elif src == "rmse":
        mat = rmse_all
        ylab = "RMSE (a.u.)"
    elif src == "nrmse":
        mat = nrmse_all
        ylab = "NRMSE (unitless)"
    elif src == "Q":
        mat = Q_all
        ylab = "Q (π f τ)"
    else:
        raise ValueError("Unknown selection")

    # --- label mask: (all) / Outlier / specific label ---
    valid_lbl = ~np.isin(labels_str, ["", "nan", "None"])

    if label_filter == "(all)":
        label_mask = np.ones_like(labels_str, dtype=bool)
        label_info = "all labels"
    elif label_filter == "Outlier":
        good_mask = np.isin(labels_str, list(GOOD_LABELS))
        label_mask = valid_lbl & ~good_mask
        label_info = f"outliers (labels not in {sorted(GOOD_LABELS)})"
    else:
        label_mask = (labels_str == label_filter)
        label_info = f"label='{label_filter}'"

    # --- 1) scatter ---
    plt.figure(figsize=(10, 5))
    for i in range(start, end + 1):
        j_idx = np.where(label_mask[i, :])[0]
        if j_idx.size == 0:
            continue
        label = str(nozzles[i]) if (end - start) < 15 else None
        plt.scatter(pressures[j_idx], mat[i, j_idx], alpha=0.3, s=20, marker="o", label=label)

    plt.title(f"{dataset} — {name} vs Pressure (Nozzles {start}-{end}, {label_info})")
    plt.xlabel("Pressure (mBar)")
    plt.ylabel(ylab)
    plt.grid(True)
    if (end - start) < 15:
        plt.legend(loc="best", fontsize=8)
    plt.tight_layout()
    plt.show()

    # --- 2) mean ± std ---
    subset   = mat[start:end+1, :]
    lbl_sub  = label_mask[start:end+1, :]
    subset_m = np.where(lbl_sub, subset, np.nan)

    mean_v = np.nanmean(subset_m, axis=0)
    std_v  = np.nanstd(subset_m, axis=0)

    plt.figure(figsize=(10, 5))
    plt.fill_between(pressures, mean_v - std_v, mean_v + std_v, alpha=0.3, label="Mean ± 1 STD")
    plt.plot(pressures, mean_v, lw=2, label="Mean")
    plt.title(f"{dataset} — Mean ± STD of {name} (Nozzles {start}-{end}, {label_info})")
    plt.xlabel("Pressure (mBar)")
    plt.ylabel(ylab)
    plt.grid(True)
    plt.legend(loc="best", fontsize=8)
    plt.tight_layout()
    plt.show()


# --- widgets ---
dataset_dropdown = Dropdown(options=DATASETS, value="1_lowering", description="Dataset:")

param_dropdown = Dropdown(
    options=[(v[0], k) for k, v in selector.items()],
    value="A",
    description="Parameter:"
)

# placeholder max; will be corrected after first load via STATE
range_slider = IntRangeSlider(
    value=[0, 9],
    min=0, max=1279, step=1,
    description="Nozzles:", continuous_update=False
)

use_khz_cb = Checkbox(value=True, description="f in kHz")
use_us_cb  = Checkbox(value=True, description="τ in µs")

label_dropdown = Dropdown(
    options=["(all)", "Outlier"],  # will be updated after load
    value="(all)",
    description="Label:"
)

def _sync_ui(dataset):
    # Load once to get correct bounds + labels
    info = load_merged_for_dataset(dataset)
    STATE.clear()
    STATE.update(info)
    STATE["dataset"] = dataset

    range_slider.max = info["n_nozzles"] - 1
    range_slider.value = [0, min(9, info["n_nozzles"] - 1)]
    label_dropdown.options = ["(all)", "Outlier"] + info["label_list"]
    label_dropdown.value = "(all)"

# initialize UI
_sync_ui(dataset_dropdown.value)

def _on_dataset_change(change):
    _sync_ui(change["new"])

dataset_dropdown.observe(_on_dataset_change, names="value")

interact(
    plot_param,
    dataset=dataset_dropdown,
    which=param_dropdown,
    nozzle_range=range_slider,
    use_kHz=use_khz_cb,
    use_us=use_us_cb,
    label_filter=label_dropdown
)



 Loaded merged arrays from: /content/drive/MyDrive/data/fit/merged_rep_1_lowering_bankcorrected_params6_Q_rmse_nrmse_labels.npz
params6: (1280, 95, 6)  rmse: (1280, 95)  nrmse: (1280, 95)  Q: (1280, 95)
labels:  (1280, 95)
Available labels: Deviated, Intermittent, Jetting, Non-jetting, Other


interactive(children=(Dropdown(description='Dataset:', options=('1_lowering', '2_lowering', '1_increasing'), v…

<function __main__.plot_param(dataset='1_lowering', which='A', nozzle_range=(0, 9), use_kHz=True, use_us=True, label_filter='(all)')>


 Loaded merged arrays from: /content/drive/MyDrive/data/fit/merged_rep_2_lowering_bankcorrected_params6_Q_rmse_nrmse_labels.npz
params6: (1280, 95, 6)  rmse: (1280, 95)  nrmse: (1280, 95)  Q: (1280, 95)
labels:  (1280, 95)
Available labels: (none)


**Outlier Count Summary by Pressure & Label**

In [ ]:
# --- Summary of label-based outliers by pressure and label (dataset + label dropdown, NO CSV) ---
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import ipywidgets as widgets
from IPython.display import display

FIT_DIR = "/content/drive/MyDrive/data/fit"
DATASETS = ["1_lowering", "2_lowering", "1_increasing"]

GOOD_LABELS = {"Jetting"}      # "normal"
SMALL_UNIQUE_THRESH = 10       # print pressures with unique_count <= this

def load_labels_df(dataset_key: str):
    rep = int(dataset_key.split("_")[0])
    ver = dataset_key.split("_", 1)[1]

    merged_fp = os.path.join(
        FIT_DIR,
        f"merged_rep_{rep}_{ver}_bankcorrected_params6_Q_rmse_nrmse_labels.npz"
    )
    if not os.path.exists(merged_fp):
        raise FileNotFoundError(f"Missing merged+labels NPZ:\n  {merged_fp}")

    m = np.load(merged_fp, allow_pickle=True)

    labels_all = m["labels"]  # (n_nozzles, n_pressures)
    pressures  = m["pressures"].astype(float)
    nozzles    = m["nozzles"]
    inv        = m.get("inverted_mask", None)

    n_nozzles, n_pressures = labels_all.shape

    def bank_from_nozzle(i):
        s = str(nozzles[i])
        return s[0].upper() if s else "?"

    rows = []
    for i in range(n_nozzles):
        for j in range(n_pressures):
            lbl = str(labels_all[i, j]).strip()
            if lbl in ("", "nan", "None"):
                continue
            row = {
                "nozzle_idx": int(i),
                "nozzle_id": str(nozzles[i]),
                "bank": bank_from_nozzle(i),
                "pressure_idx": int(j),
                "pressure_mBar": float(pressures[j]),
                "label": lbl,
            }
            if inv is not None:
                row["inverted_flag"] = bool(inv[i, j])
            rows.append(row)

    labels_df = pd.DataFrame(rows)
    meta = dict(
        merged_fp=merged_fp,
        n_nozzles=len(nozzles),
        n_pressures=len(pressures),
        total_signals=len(nozzles) * len(pressures),
    )
    return labels_df, meta


def summarize_and_plot(dataset_key: str, label_choice: str):
    labels_df, meta = load_labels_df(dataset_key)

    if labels_df.empty:
        print(f"[{dataset_key}] No labels found.")
        return

    # available labels in this dataset
    available_labels = sorted(labels_df["label"].unique().tolist())

    # filter based on dropdown
    if label_choice == "(all)":
        df_sel = labels_df.copy()
        subtitle = "all labels"
    elif label_choice == "Outlier":
        df_sel = labels_df[~labels_df["label"].isin(GOOD_LABELS)].copy()
        subtitle = f"outliers (labels not in {sorted(GOOD_LABELS)})"
    else:
        df_sel = labels_df[labels_df["label"] == label_choice].copy()
        subtitle = f"label='{label_choice}'"

    # --- print compact counts (top-level) ---
    # 1) total label counts (all)
    counts_all = labels_df["label"].value_counts()
    # 2) selection size
    sel_n = len(df_sel)

    print(f"\n[{dataset_key}] merged: {meta['merged_fp']}")
    print(f"[{dataset_key}] labels present: " + " | ".join([f"{k}={int(v)}" for k, v in counts_all.items()]))
    print(f"[{dataset_key}] selection: {subtitle} -> {sel_n} rows")

    if df_sel.empty:
        print("Selection is empty — nothing to plot.")
        return

    # --- Outlier-like summaries (works for any label selection) ---
    by_pressure = (
        df_sel.groupby("pressure_mBar", dropna=False)
        .size().reset_index(name="count")
        .sort_values("pressure_mBar")
    )

    by_label = (
        df_sel.groupby("label", dropna=False)
        .size().reset_index(name="count")
        .sort_values("count", ascending=False)
    )

    unique_nozzles = df_sel["nozzle_idx"].nunique()
    nozzle_fraction = unique_nozzles / max(1, meta["n_nozzles"])

    pair_count = df_sel[["nozzle_idx", "pressure_idx"]].drop_duplicates().shape[0]
    pair_fraction = pair_count / max(1, meta["total_signals"])

    print("\n🔹 By pressure (first 10):")
    print(by_pressure.head(10).to_string(index=False))

    print("\n🔹 By label:")
    print(by_label.to_string(index=False))

    print(f"\n🔹 Unique nozzles flagged: {unique_nozzles}/{meta['n_nozzles']} ({nozzle_fraction*100:.2f}%)")
    print(f"🔹 Unique (nozzle×pressure) pairs: {pair_count}/{meta['total_signals']} ({pair_fraction*100:.2f}%)")

    # --- Plot: unique (nozzle×pressure) per pressure (deduped) ---
    dedup_pairs = (
        df_sel[["nozzle_idx", "pressure_idx", "pressure_mBar"]]
        .drop_duplicates()
        .sort_values(["pressure_mBar", "nozzle_idx", "pressure_idx"])
    )

    unique_by_pressure = (
        dedup_pairs.groupby("pressure_mBar", dropna=False)
        .size().reset_index(name="unique_count")
        .sort_values("pressure_mBar")
    )

    plt.figure(figsize=(8, 4))
    plt.plot(unique_by_pressure["pressure_mBar"], unique_by_pressure["unique_count"], "o-", lw=2)
    plt.title(f"{dataset_key} — Unique signals vs Pressure\n({subtitle}, bank-corrected)")
    plt.xlabel("Pressure (mBar)")
    plt.ylabel("Count of unique nozzle×pressure pairs")
    plt.grid(True, alpha=0.4)
    plt.tight_layout()
    plt.show()

    tail = unique_by_pressure[unique_by_pressure["unique_count"] <= SMALL_UNIQUE_THRESH]
    if not tail.empty:
        print(f"\nPressures with small unique-counts (<= {SMALL_UNIQUE_THRESH}):")
        print(tail.to_string(index=False))


# --- UI ---
ds_dropdown = widgets.Dropdown(options=DATASETS, value="1_lowering", description="Dataset:")
label_dropdown = widgets.Dropdown(options=["(all)", "Outlier"], value="(all)", description="Label:")

out = widgets.Output()

def refresh_label_options(dataset_key: str):
    labels_df, _ = load_labels_df(dataset_key)
    opts = ["(all)", "Outlier"] + sorted(labels_df["label"].unique().tolist()) if not labels_df.empty else ["(all)", "Outlier"]
    label_dropdown.options = opts
    if label_dropdown.value not in opts:
        label_dropdown.value = "(all)"

def rerun():
    with out:
        out.clear_output(wait=True)
        summarize_and_plot(ds_dropdown.value, label_dropdown.value)

def on_dataset_change(change):
    if change["name"] == "value":
        refresh_label_options(change["new"])
        rerun()

def on_label_change(change):
    if change["name"] == "value":
        rerun()

ds_dropdown.observe(on_dataset_change, names="value")
label_dropdown.observe(on_label_change, names="value")

display(widgets.HBox([ds_dropdown, label_dropdown]), out)

# initial
refresh_label_options(ds_dropdown.value)
rerun()


Output()

**Parameters boxplots by label**

In [ ]:
# ==== Boxplots: fitted params only, per-label (Jetting + individual non-jetting labels)
#      + dataset dropdown (NO Outlier boxplot; Intermittent before Deviated)
#      + FIX: prevents double plotting on init (no interact) ====

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ipywidgets as widgets
from IPython.display import display

sns.set(style="whitegrid")

FIT_DIR     = "/content/drive/MyDrive/data/fit"
DATASETS    = ["1_lowering", "2_lowering", "1_increasing"]
GOOD_LABELS = {"Jetting"}

FEATURE_COLS_ALL = ["fit_A","fit_tau","fit_f","fit_phi","fit_C","fit_D","fit_Q","fit_rmse"]

def assign_bank_from_name(nozzle_id) -> str:
    s = str(nozzle_id)
    return s[0].upper() if s else "?"

def load_label_box_df(dataset_key: str):
    rep = int(dataset_key.split("_")[0])
    ver = dataset_key.split("_", 1)[1]

    merged_fp = os.path.join(
        FIT_DIR,
        f"merged_rep_{rep}_{ver}_bankcorrected_params6_Q_rmse_nrmse_labels.npz"
    )
    if not os.path.exists(merged_fp):
        raise FileNotFoundError(f"Missing merged NPZ with labels:\n{merged_fp}")

    m = np.load(merged_fp, allow_pickle=True)
    pressures    = m["pressures"].astype(float)
    nozzles      = m["nozzles"]
    params6_all  = m["params6_all"]   # (N, P, 6)
    rmse_all     = m["rmse_all"]      # (N, P)
    labels_raw   = m["labels"]        # (N, P)

    N, P, _ = params6_all.shape

    # flatten params
    idx = pd.MultiIndex.from_product([range(N), range(P)], names=["NozzleIdx","PressureIdx"])
    p6_flat = params6_all.reshape(-1, 6)
    rm_flat = rmse_all.reshape(-1)

    param_df = pd.DataFrame(
        p6_flat, columns=["fit_A","fit_tau","fit_f","fit_phi","fit_C","fit_D"]
    )
    param_df["fit_rmse"]    = rm_flat
    param_df["fit_Q"]       = np.pi * param_df["fit_f"] * param_df["fit_tau"]
    param_df["NozzleIdx"]   = idx.get_level_values("NozzleIdx").astype(int)
    param_df["PressureIdx"] = idx.get_level_values("PressureIdx").astype(int)

    # attach ids
    param_df["NozzleID"]      = param_df["NozzleIdx"].apply(lambda i: nozzles[i])
    param_df["Pressure_mBar"] = param_df["PressureIdx"].apply(lambda j: pressures[j])
    param_df["Bank"]          = param_df["NozzleID"].apply(assign_bank_from_name)

    # labels
    labels_str  = np.char.strip(labels_raw.astype(str))
    labels_flat = labels_str.reshape(-1)
    param_df["LabelRaw"] = pd.Series(labels_flat).replace({"": np.nan, "nan": np.nan, "None": np.nan})

    feature_cols = [c for c in FEATURE_COLS_ALL if c in param_df.columns]

    # non-jetting labels present (excluding NaN)
    non_jet_labels = sorted(
        lbl for lbl in param_df["LabelRaw"].dropna().unique()
        if lbl not in GOOD_LABELS
    )

    # build per-label box df (NO Outlier aggregation)
    cols_keep = ["NozzleIdx","PressureIdx","NozzleID","Pressure_mBar","Bank"] + feature_cols + ["LabelRaw"]

    rows_box = []
    for _, row in param_df[cols_keep].iterrows():
        lbl = row["LabelRaw"]
        if pd.isna(lbl):
            continue
        base = row.to_dict()
        base.pop("LabelRaw", None)

        if lbl in GOOD_LABELS:
            r = base.copy()
            r["Group"] = "Jetting"
            rows_box.append(r)
        else:
            r = base.copy()
            r["Group"] = str(lbl)   # ONLY the specific label
            rows_box.append(r)

    label_box_df = pd.DataFrame(rows_box)
    return label_box_df, feature_cols, non_jet_labels


# ---- cache (avoid reload on every tick) ----
CACHE = {"dataset": None, "label_box_df": None, "feature_cols": None, "non_jet_labels": None}

def plot_feature_boxplot_all(dataset, feature, bank):
    # load dataset if needed
    if CACHE["dataset"] != dataset:
        label_box_df, feature_cols, non_jet_labels = load_label_box_df(dataset)
        CACHE.update({
            "dataset": dataset,
            "label_box_df": label_box_df,
            "feature_cols": feature_cols,
            "non_jet_labels": non_jet_labels
        })

    df = CACHE["label_box_df"]
    non_jet_labels = CACHE["non_jet_labels"]

    if df is None or df.empty:
        plt.figure(figsize=(6,2))
        plt.axis("off")
        plt.text(0.5, 0.5, f"No labeled data available for {dataset}.", ha="center", va="center")
        plt.show()
        plt.close()
        return

    # bank filter
    if bank != "All":
        dfp = df[df["Bank"] == bank]
        title_bank = f"Bank {bank}"
    else:
        dfp = df
        title_bank = "All banks"

    if dfp.empty:
        plt.figure(figsize=(6,2))
        plt.axis("off")
        plt.text(0.5, 0.5, f"No data for {dataset} — {title_bank}.", ha="center", va="center")
        plt.show()
        plt.close()
        return

    # ---- ORDER: Jetting first, then Intermittent, then Deviated, then the rest ----
    present = set(dfp["Group"].unique().tolist())

    order = []
    if "Jetting" in present:
        order.append("Jetting")

    for forced in ["Intermittent", "Deviated"]:
        if forced in present and forced not in order:
            order.append(forced)

    forced_set = set(order)
    for lbl in non_jet_labels:
        if lbl in present and lbl not in forced_set:
            order.append(lbl)

    plt.figure(figsize=(max(8, 0.8 * len(order)), 5))
    sns.boxplot(
        data=dfp,
        x="Group",
        y=feature,
        order=order,
        showfliers=False,
    )
    plt.title(f"{dataset} — {feature} — {title_bank}\nJetting + individual non-jetting labels (no Outlier)")
    plt.ylabel(feature); plt.xlabel("")
    plt.xticks(rotation=30, ha="right")
    plt.grid(axis="y", alpha=0.3)
    plt.tight_layout()
    plt.show()
    plt.close()


# =========================
# UI (single-output, no double plot)
# =========================
dataset_w = widgets.Dropdown(options=DATASETS, value=DATASETS[0], description="Dataset:")
bank_w    = widgets.Dropdown(options=["All", "A", "B", "C", "D"], value="All", description="Bank:")

# init once to get feature list
_init_df, _init_features, _init_nonjet = load_label_box_df(DATASETS[0])
CACHE.update({
    "dataset": DATASETS[0],
    "label_box_df": _init_df,
    "feature_cols": _init_features,
    "non_jet_labels": _init_nonjet
})

feature_w = widgets.Dropdown(
    options=CACHE["feature_cols"],
    value=("fit_A" if "fit_A" in CACHE["feature_cols"] else CACHE["feature_cols"][0]),
    description="Feature:"
)

out = widgets.Output()
UPDATING = False

def refresh_dataset(ds_key: str):
    """Reload cache for dataset and update feature dropdown options safely."""
    global UPDATING
    UPDATING = True

    label_box_df, feature_cols, non_jet_labels = load_label_box_df(ds_key)
    CACHE.update({
        "dataset": ds_key,
        "label_box_df": label_box_df,
        "feature_cols": feature_cols,
        "non_jet_labels": non_jet_labels
    })

    feature_w.options = feature_cols
    if feature_w.value not in feature_cols:
        feature_w.value = ("fit_A" if "fit_A" in feature_cols else feature_cols[0])

    UPDATING = False

def render(_=None):
    if UPDATING:
        return
    with out:
        out.clear_output(wait=True)
        plot_feature_boxplot_all(dataset_w.value, feature_w.value, bank_w.value)

def on_dataset_change(change):
    if change["name"] == "value":
        refresh_dataset(change["new"])
        render()

dataset_w.observe(on_dataset_change, names="value")
feature_w.observe(render, names="value")
bank_w.observe(render, names="value")

display(widgets.HBox([dataset_w, feature_w, bank_w]), out)
render()


Output()

**Check for “non-jet → jetting again” as pressure lowers**

In [ ]:
# ==== Jetting → X → Jetting episodes (ALL datasets) — direction-aware (minimal change) ====
import numpy as np
import os

FIT_DIR   = "/content/drive/MyDrive/data/fit"
DATASETS  = ["1_lowering", "2_lowering", "1_increasing"]   # add more if needed

GOOD_LABEL = "jetting"
EMPTY_TOKENS = {"", "nan", "NaN", "NONE", "None", "none"}
nonjet_variants = ("non-jetting", "non jetting", "nonjetting", "non_jetting")

def canonical_label(s: str) -> str:
    s = str(s).strip()
    low = s.lower()
    if low in nonjet_variants:
        return "non-jetting"
    return low

def load_labels_npz(dataset_key: str):
    rep = int(dataset_key.split("_")[0])
    ver = dataset_key.split("_", 1)[1]
    fp = os.path.join(
        FIT_DIR,
        f"merged_rep_{rep}_{ver}_bankcorrected_params6_Q_rmse_nrmse_labels.npz"
    )
    if not os.path.exists(fp):
        raise FileNotFoundError(f"Missing merged NPZ with labels:\n  {fp}")
    m = np.load(fp, allow_pickle=True)
    pressures  = m["pressures"].astype(float)
    labels_raw = m["labels"]
    return fp, pressures, labels_raw

def idx_in_experiment_direction(dataset_key: str, pressures: np.ndarray):
    # minimal correction:
    # - lowering:  walk high -> low (same as before)
    # - increasing: walk low -> high (so "later" means later in increasing run)
    if "increasing" in dataset_key.lower():
        return np.argsort(pressures)          # low -> high
    else:
        return np.argsort(pressures)[::-1]    # high -> low

def compute_revert_stats(dataset_key: str, pressures, labels_raw):
    labels_str = np.char.strip(labels_raw.astype(str))
    for bad in EMPTY_TOKENS:
        labels_str[labels_str == bad] = ""
    labels_lower = np.char.lower(labels_str)

    N, P = labels_lower.shape

    idx_desc = idx_in_experiment_direction(dataset_key, pressures)

    total_transitions = 0
    nozzles_with_any = set()
    per_label_trans = {}
    per_label_nozz  = {}

    for i in range(N):
        lbl_seq  = labels_lower[i, idx_desc]
        orig_seq = labels_str[i, idx_desc]

        is_jet = (lbl_seq == GOOD_LABEL)
        is_lab = (lbl_seq != "")
        is_bad = is_lab & (~is_jet)

        if (not np.any(is_jet)) or (not np.any(is_lab)) or (not np.any(is_bad)):
            continue

        j = 0
        while j < P:
            if not is_bad[j]:
                j += 1
                continue

            start = j
            while j + 1 < P and is_bad[j + 1]:
                j += 1
            end = j

            has_jet_before = np.any(is_jet[:start])
            has_jet_after  = np.any(is_jet[end+1:])

            if has_jet_before and has_jet_after:
                defect_label = canonical_label(orig_seq[start])
                total_transitions += 1
                nozzles_with_any.add(i)

                per_label_trans[defect_label] = per_label_trans.get(defect_label, 0) + 1
                per_label_nozz.setdefault(defect_label, set()).add(i)

            j += 1

    per_label_nozz_count = {k: len(v) for k, v in per_label_nozz.items()}
    return {
        "N": N,
        "P": P,
        "total_transitions": total_transitions,
        "nozzles_with_any": len(nozzles_with_any),
        "per_label_trans": dict(sorted(per_label_trans.items())),
        "per_label_nozz": dict(sorted(per_label_nozz_count.items())),
    }

def fmt_counts(per_label_trans, per_label_nozz):
    parts = []
    for lab in per_label_trans.keys():
        parts.append(f"{lab}={per_label_trans[lab]}({per_label_nozz.get(lab,0)})")
    return " | ".join(parts) if parts else "(none)"

for ds in DATASETS:
    fp, pressures, labels_raw = load_labels_npz(ds)
    stats = compute_revert_stats(ds, pressures, labels_raw)

    print(
        f"[{ds}] Jet→X→Jet: {stats['total_transitions']} transitions, "
        f"{stats['nozzles_with_any']} nozzles | "
        f"breakdown: {fmt_counts(stats['per_label_trans'], stats['per_label_nozz'])}"
    )


[1_lowering] Jet→X→Jet: 839 transitions, 552 nozzles | breakdown: deviated=577(398) | intermittent=258(222) | non-jetting=1(1) | other=3(1)
[2_lowering] Jet→X→Jet: 1131 transitions, 617 nozzles | breakdown: deviated=910(516) | intermittent=218(193) | non-jetting=1(1) | other=2(1)
[1_increasing] Jet→X→Jet: 80 transitions, 75 nozzles | breakdown: deviated=71(66) | intermittent=9(9)


**Boxplots a couple steps before non.jetting on set**

In [ ]:
# ==== Boxplots: params split by label-based transition to non-jetting ====
# + Dataset dropdown (minimal fix: idx_desc respects increasing vs lowering)
# + Plot title includes selected dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import ipywidgets as widgets
from ipywidgets import Dropdown, Checkbox, BoundedIntText, HBox, VBox
from IPython.display import display

plt.style.use("default")

FIT_DIR   = "/content/drive/MyDrive/data/fit"
DATASETS  = ["1_lowering", "2_lowering", "1_increasing"]

# ---------------- Global state ----------------
dataset_key = None

replicate = None
variant   = None
MERGED_FP = None

pressures   = None
nozzles     = None
params6_all = None
rmse_all    = None
labels_raw  = None

N = P = None
idx_desc = None
has_revert = None

labels_str   = None
labels_lower = None
labels_slim  = None

param_df = None
feature_cols = None

def idx_in_experiment_direction(variant: str, pressures: np.ndarray):
    # minimal correction:
    # lowering:  high -> low (same as before)
    # increasing: low -> high
    if "increasing" in str(variant).lower():
        return np.argsort(pressures)          # low -> high
    else:
        return np.argsort(pressures)[::-1]    # high -> low

def load_dataset_state(ds_key: str):
    global dataset_key
    global replicate, variant, MERGED_FP
    global pressures, nozzles, params6_all, rmse_all, labels_raw
    global N, P, idx_desc, has_revert
    global labels_str, labels_lower, labels_slim
    global param_df, feature_cols

    dataset_key = ds_key

    replicate = int(ds_key.split("_")[0])
    variant   = ds_key.split("_", 1)[1]

    MERGED_FP = os.path.join(
        FIT_DIR,
        f"merged_rep_{replicate}_{variant}_bankcorrected_params6_Q_rmse_nrmse_labels.npz"
    )
    if not os.path.exists(MERGED_FP):
        raise FileNotFoundError(f"Missing merged NPZ with labels: {MERGED_FP}")

    m = np.load(MERGED_FP, allow_pickle=True)
    pressures    = m["pressures"].astype(float)
    nozzles      = m["nozzles"]
    params6_all  = m["params6_all"]
    rmse_all     = m["rmse_all"]
    labels_raw   = m["labels"]

    N, P, _ = params6_all.shape
    print(f" Merged NPZ loaded: {ds_key} | N={N}, P={P}, params6={params6_all.shape}, labels={labels_raw.shape}")

    # ---------------- Clean labels (STRICT Jetting / Non-jetting only) ----------------
    labels_str = np.char.strip(labels_raw.astype(str))
    for bad in ("", "nan", "NaN", "NONE", "None", "none"):
        labels_str[labels_str == bad] = ""

    labels_lower = np.char.lower(labels_str)

    labels_slim = np.empty_like(labels_lower)
    labels_slim[:] = ""
    labels_slim[labels_lower == "jetting"] = "jetting"
    nonjet_variants = ("non-jetting", "non jetting", "nonjetting", "non_jetting")
    labels_slim[np.isin(labels_lower, nonjet_variants)] = "nonjetting"

    print("\n Label normalization result (STRICT Jetting / Non-jetting only):")
    print(f"   Jetting:     {int((labels_slim == 'jetting').sum())}")
    print(f"   Non-jetting: {int((labels_slim == 'nonjetting').sum())}")
    print(f"   Blank/other: {int((labels_slim == '').sum())}")

    # ---------------- direction-aware idx_desc ----------------
    idx_desc = idx_in_experiment_direction(variant, pressures)

    # ---------------- Detect re-jetting pattern (same logic as before) ----------------
    has_revert = np.zeros(N, dtype=bool)
    for i in range(N):
        lbl_seq = labels_slim[i, idx_desc]
        is_jet  = (lbl_seq == "jetting")
        is_non  = (lbl_seq == "nonjetting")

        if not np.any(is_non) or not np.any(is_jet):
            continue

        first_non_pos = np.where(is_non)[0][0]
        if np.any(is_jet[first_non_pos+1:]):
            has_revert[i] = True

    n_revert = int(has_revert.sum())
    print(f"\n Nozzles with Non-jetting → Jetting-again pattern: {n_revert} / {N}")
    print(f" Nozzles with monotone behaviour (no revert):        {N - n_revert} / {N}\n")

    # ---------------- Build tidy DataFrame of fitted params (same) ----------------
    idx = pd.MultiIndex.from_product([range(N), range(P)], names=["NozzleIdx","PressureIdx"])
    p6_flat  = params6_all.reshape(-1, 6)
    rm_flat  = rmse_all.reshape(-1)

    param_df = pd.DataFrame(p6_flat, columns=["fit_A","fit_tau","fit_f","fit_phi","fit_C","fit_D"])
    param_df["fit_rmse"]     = rm_flat
    param_df["NozzleIdx"]    = idx.get_level_values("NozzleIdx").astype(int)
    param_df["PressureIdx"]  = idx.get_level_values("PressureIdx").astype(int)
    param_df["fit_Q"]        = np.pi * param_df["fit_f"] * param_df["fit_tau"]

    param_df["Pressure_mBar"] = param_df["PressureIdx"].apply(lambda j: pressures[j])
    param_df["NozzleID"]      = param_df["NozzleIdx"].apply(lambda i: nozzles[i])
    param_df["HasRevert"]     = param_df["NozzleIdx"].map({i: bool(has_revert[i]) for i in range(N)})

    feature_cols = ["fit_A", "fit_tau", "fit_f", "fit_phi", "fit_C", "fit_D", "fit_Q", "fit_rmse"]
    feature_cols = [f for f in feature_cols if f in param_df.columns]
    print(f" Fitted-parameter features available: {feature_cols}\n")


def build_transition_groups(steps_before: int):
    steps_before = int(max(0, steps_before))
    label_pre = f"{steps_before} steps before onset"
    transition_group = np.empty((N, P), dtype=object)

    for i in range(N):
        lbl_seq = labels_slim[i, idx_desc]
        is_non  = (lbl_seq == "nonjetting")

        non_pos = np.where(is_non)[0]
        if non_pos.size > 0:
            first_non_pos = non_pos[0]
            start_pre = max(0, first_non_pos - steps_before)

            if start_pre > 0:
                transition_group[i, idx_desc[:start_pre]] = "Earlier / never non-jet"
            if first_non_pos > start_pre:
                transition_group[i, idx_desc[start_pre:first_non_pos]] = label_pre
            transition_group[i, idx_desc[first_non_pos:]] = "After non-jetting onset"
        else:
            transition_group[i, :] = "Earlier / never non-jet"

    return transition_group, ["After non-jetting onset", label_pre, "Earlier / never non-jet"]


def plot_feature_by_transition_group(feature, include_reverts=True, steps_before=1):
    transition_group, group_order = build_transition_groups(steps_before)

    df = param_df.copy()
    df["TransitionGroup"] = transition_group.reshape(-1)

    if not include_reverts:
        df = df[df["HasRevert"] == False].copy()
        defect_nozzles = df.loc[df["TransitionGroup"] == "After non-jetting onset", "NozzleIdx"].unique()
        df = df[df["NozzleIdx"].isin(defect_nozzles)]

    plt.figure(figsize=(8,5))
    data = [df.loc[df["TransitionGroup"] == g, feature].dropna().values for g in group_order]

    labels_non_empty = [lab for lab, arr in zip(group_order, data) if len(arr) > 0]
    data_non_empty   = [arr for arr in data if len(arr) > 0]

    if not data_non_empty:
        print(" No data in any transition group (after filtering).")
        plt.close()
        return

    bp = plt.boxplot(
        data_non_empty,
        tick_labels=labels_non_empty,
        patch_artist=True,
        showfliers=False
    )

    for box in bp["boxes"]:
        box.set(facecolor="blue", alpha=0.4, edgecolor="blue", linewidth=1.5)
    for whisker in bp["whiskers"]:
        whisker.set(color="blue", linewidth=1.2)
    for cap in bp["caps"]:
        cap.set(color="blue", linewidth=1.2)
    for median in bp["medians"]:
        median.set(color="blue", linewidth=1.5)

    plt.ylabel(feature)
    plt.xlabel("Transition group (relative to first Non-jetting label)")
    suffix = " (all nozzles)" if include_reverts else " (excluding re-jetting nozzles)"
    plt.title(f"{dataset_key} — {feature}\nDistribution across 3 transition groups{suffix}")
    plt.grid(axis="y", alpha=0.3)
    plt.tight_layout()
    plt.show()


# =========================
# UI (no double-plot)
# =========================
dataset_w = Dropdown(options=DATASETS, value=DATASETS[0], description="Dataset:")
feature_w = Dropdown(options=[], description="Parameter:")
include_w = Checkbox(value=True, description="Include re-jetting nozzles?")
steps_w   = BoundedIntText(value=1, min=0, max=1, description="Steps before onset:")

out_info = widgets.Output()
out_plot = widgets.Output()
UPDATING = False

def refresh_for_dataset():
    global UPDATING
    UPDATING = True
    with out_info:
        out_info.clear_output(wait=True)
        load_dataset_state(dataset_w.value)

    feature_w.options = feature_cols
    if feature_w.value not in feature_cols:
        feature_w.value = ("fit_A" if "fit_A" in feature_cols else feature_cols[0])

    steps_w.max = int(P)
    if steps_w.value > steps_w.max:
        steps_w.value = steps_w.max

    UPDATING = False

def run_plot(_=None):
    if UPDATING:
        return
    with out_plot:
        out_plot.clear_output(wait=True)
        plot_feature_by_transition_group(
            feature=feature_w.value,
            include_reverts=include_w.value,
            steps_before=steps_w.value
        )

def on_dataset_change(change):
    if change["name"] == "value":
        refresh_for_dataset()
        run_plot()

dataset_w.observe(on_dataset_change, names="value")
feature_w.observe(run_plot, names="value")
include_w.observe(run_plot, names="value")
steps_w.observe(run_plot, names="value")

refresh_for_dataset()
controls = HBox([dataset_w, feature_w, include_w, steps_w])
display(VBox([controls, out_info, out_plot]))
run_plot()


**Interactive map of nozzle labels**

In [ ]:
# === Interactive nozzle label map (Plotly)
#     + Dataset dropdown (1_lowering / 2_lowering / 1_increasing)
#     + Pressure dropdown LIMITED to pressures that actually contain labels (i.e. not all "Unlabeled")
#     + Prints compact (linear) label counts at the selected pressure
#     + Keeps Jetting=green, NonJetting=red
#     + Puts dataset name in the plot title
#
# FIX: use display(fig) instead of fig.show() so the figure refreshes correctly inside the Output widget.

import os, re
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

import ipywidgets as widgets
from IPython.display import display, clear_output

# ---------------- Render fix (Colab + Jupyter) ----------------
try:
    from google.colab import output as colab_output
    colab_output.enable_custom_widget_manager()
    pio.renderers.default = "colab"
except Exception:
    # This keeps the SAME figure representation, but makes updates reliable inside widgets.Output
    pio.renderers.default = "plotly_mimetype"

# ---------------- Config ----------------
FIT_DIR   = "/content/drive/MyDrive/data/fit"
DATASETS  = ["1_lowering", "2_lowering", "1_increasing"]

# ---------------- Canonical nozzle parsing: 'NA_1' -> ('A', 1) ----------------
def canonical_nozzle(name):
    s = str(name).strip()
    m = re.search(r"([ABCD])[^0-9]*([0-9]+)", s)
    if not m:
        return None, None
    return m.group(1), int(m.group(2))

# ---------------- Label canonicalization ----------------
nonjet_variants = ("non-jetting", "non jetting", "nonjetting", "non_jetting")

def canonical_label(raw):
    s = str(raw).strip()
    if s == "" or s.lower() in ("nan", "none"):
        return "Unlabeled"

    s_low = s.lower()
    if s_low == "jetting":
        return "Jetting"
    if s_low in nonjet_variants:
        return "NonJetting"
    if s_low == "other":
        return "Other"

    return s  # keep as-is

# ---------------- State container ----------------
STATE = {}

def load_dataset(dataset_key: str):
    """Load merged+labels NPZ and rebuild all derived structures for plotting."""
    rep = int(dataset_key.split("_")[0])
    ver = dataset_key.split("_", 1)[1]

    merged_fp = os.path.join(
        FIT_DIR,
        f"merged_rep_{rep}_{ver}_bankcorrected_params6_Q_rmse_nrmse_labels.npz"
    )
    if not os.path.exists(merged_fp):
        raise FileNotFoundError(f"Missing merged NPZ with labels:\n{merged_fp}")

    m = np.load(merged_fp, allow_pickle=True)
    labels_all = m["labels"]                # (N,P)
    pressures  = m["pressures"].astype(float)
    nozzles    = m["nozzles"].astype(object)

    N, P = labels_all.shape

    # --- Discover banks + max nozzle number per bank ---
    banks = []
    max_noz_per_bank = {}
    for n in nozzles:
        bank, num = canonical_nozzle(n)
        if bank is None:
            continue
        if bank not in banks:
            banks.append(bank)
        max_noz_per_bank[bank] = max(max_noz_per_bank.get(bank, 0), num)

    banks = [b for b in "ABCD" if b in banks]  # stable order
    bank_to_idx = {b: i for i, b in enumerate(banks)}

    # --- Pre-compute map coordinates (row, col) for each nozzle ---
    row_of_nozzle = np.full(N, np.nan, dtype=float)
    col_of_nozzle = np.full(N, np.nan, dtype=float)

    for i, name in enumerate(nozzles):
        bank, num = canonical_nozzle(name)
        if bank is None or num is None or bank not in bank_to_idx:
            continue
        bank_idx   = bank_to_idx[bank]
        phase      = (num - 1) % 4
        col        = (num - 1) // 4
        global_row = phase * 4 + bank_idx
        row_of_nozzle[i] = global_row
        col_of_nozzle[i] = col

    # --- Clean labels and precompute canonical categories ---
    labels_str = np.char.strip(labels_all.astype(str))
    for bad in ("", "nan", "None"):
        labels_str[labels_str == bad] = ""

    labels_cat = np.vectorize(canonical_label, otypes=[object])(labels_str)
    all_cats = sorted(set(labels_cat.ravel().tolist()))

    # --- Colors: keep Jetting green and NonJetting red ---
    colors_by_cat = {
        "Jetting":    "rgb(0,150,0)",
        "NonJetting": "rgb(200,0,0)",
        "Unlabeled":  "rgba(200,200,200,0.15)",
        "Other":      "rgb(0,0,200)",  # blue
    }
    extra_palette = [
        "rgb(200,120,0)",
        "rgb(120,0,120)",
        "rgb(0,160,160)",
        "rgb(160,80,0)",
        "rgb(80,0,160)",
        "rgb(0,120,80)",
        "rgb(200,0,150)",
    ]
    palette_idx = 0
    for cat in all_cats:
        if cat not in colors_by_cat:
            colors_by_cat[cat] = extra_palette[palette_idx % len(extra_palette)]
            palette_idx += 1

    # --- Order categories in legend ---
    ordered_categories = []
    for base in ["Jetting", "NonJetting", "Other"]:
        if base in all_cats:
            ordered_categories.append(base)
    for cat in all_cats:
        if cat not in ordered_categories and cat != "Unlabeled":
            ordered_categories.append(cat)
    if "Unlabeled" in all_cats:
        ordered_categories.append("Unlabeled")

    # --- Row tick labels (A1..D4 style) ---
    row_labels = []
    for phase in range(4):
        for bank in banks:
            row_labels.append(f"{bank}{phase+1}")

    # --- LIMIT pressure dropdown to labelled pressures only ---
    labelled_mask = np.array([np.any(labels_cat[:, j] != "Unlabeled") for j in range(P)], dtype=bool)
    valid_press_idxs = np.where(labelled_mask)[0].tolist()
    if len(valid_press_idxs) == 0:
        valid_press_idxs = list(range(P))

    press_options = [(f"{pressures[j]:.1f} mBar (idx {j})", int(j)) for j in valid_press_idxs]

    STATE.clear()
    STATE.update(dict(
        dataset_key=dataset_key,
        merged_fp=merged_fp,
        labels_all=labels_all,
        pressures=pressures,
        nozzles=nozzles,
        N=N, P=P,
        banks=banks,
        row_of_nozzle=row_of_nozzle,
        col_of_nozzle=col_of_nozzle,
        labels_cat=labels_cat,
        colors_by_cat=colors_by_cat,
        ordered_categories=ordered_categories,
        row_labels=row_labels,
        press_options=press_options,
        valid_press_idxs=set(valid_press_idxs),
    ))

def make_label_map(press_idx: int):
    press_idx = int(press_idx)

    dataset_key = STATE["dataset_key"]
    labels_cat  = STATE["labels_cat"]
    labels_all  = STATE["labels_all"]
    pressures   = STATE["pressures"]
    nozzles     = STATE["nozzles"]
    row_of_nozzle = STATE["row_of_nozzle"]
    col_of_nozzle = STATE["col_of_nozzle"]
    colors_by_cat = STATE["colors_by_cat"]
    ordered_categories = STATE["ordered_categories"]
    row_labels = STATE["row_labels"]

    cats_for_press = labels_cat[:, press_idx]
    traces = []

    for cat in ordered_categories:
        xs, ys, hover = [], [], []

        for i, name in enumerate(nozzles):
            r = row_of_nozzle[i]
            c = col_of_nozzle[i]
            if np.isnan(r) or np.isnan(c):
                continue
            if cats_for_press[i] != cat:
                continue

            bank, num = canonical_nozzle(name)
            xs.append(c + 1)  # 1..80
            ys.append(r + 1)  # 1..16
            hover.append(
                f"Nozzle: {bank}{num}<br>"
                f"Index in array: {i}<br>"
                f"Raw label: {labels_all[i, press_idx]}<br>"
                f"Shown label: {cat}<br>"
                f"Dataset: {dataset_key}<br>"
                f"Pressure: {pressures[press_idx]:.1f} mBar (idx {press_idx})"
            )

        if len(xs) == 0:
            continue

        traces.append(
            go.Scatter(
                x=xs, y=ys,
                mode="markers",
                name=str(cat),
                marker=dict(
                    size=14,
                    color=colors_by_cat.get(cat, "rgb(0,0,0)"),
                    line=dict(width=1, color="white"),
                ),
                hovertext=hover,
                hoverinfo="text",
            )
        )

    fig = go.Figure(data=traces)
    fig.update_layout(
        title=f"{dataset_key} — Nozzle labels at {STATE['pressures'][press_idx]:.1f} mBar (idx {press_idx})",
        xaxis=dict(
            title="Column",
            range=[0.5, 80.5],
            tickmode="array",
            tickvals=list(range(1, 81, 4)),
            ticktext=[str(i) for i in range(1, 81, 4)],
            showgrid=False,
            zeroline=False,
        ),
        yaxis=dict(
            title="Row (Bank / phase)",
            range=[16.5, 0.5],
            tickmode="array",
            tickvals=[i + 1 for i in range(16)],
            ticktext=row_labels,
            showgrid=False,
            zeroline=False,
        ),
        width=1400,
        height=550,
        plot_bgcolor="white",
        paper_bgcolor="white",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        margin=dict(l=40, r=40, t=60, b=40),
    )
    return fig

# ---------------- Widgets ----------------
dataset_dd = widgets.Dropdown(options=DATASETS, value=DATASETS[0], description="Dataset:")
press_dd   = widgets.Dropdown(options=[], description="Pressure:")

out = widgets.Output()
UPDATING = False

def set_pressure_options():
    global UPDATING
    UPDATING = True
    press_dd.options = STATE["press_options"]
    if len(STATE["press_options"]) > 0:
        press_dd.value = STATE["press_options"][0][1]
    UPDATING = False

def print_counts_line(press_idx: int):
    labels_cat  = STATE["labels_cat"]
    pressures   = STATE["pressures"]
    dataset_key = STATE["dataset_key"]

    vc = pd.Series(labels_cat[:, press_idx]).value_counts()
    parts = [f"{k}={int(v)}" for k, v in vc.items()]
    print(f"[{dataset_key}] Pressure {pressures[press_idx]:.1f} mBar (idx {press_idx}) — " + " | ".join(parts))

def render(_=None):
    if UPDATING:
        return
    press_idx = int(press_dd.value)
    with out:
        clear_output(wait=True)
        print_counts_line(press_idx)
        fig = make_label_map(press_idx)
        display(fig)  # <-- IMPORTANT: replaces fig.show() so the plot refreshes inside `out`

def on_dataset_change(change):
    if change["name"] == "value":
        with out:
            clear_output(wait=True)
        load_dataset(change["new"])
        set_pressure_options()
        render()

dataset_dd.observe(on_dataset_change, names="value")
press_dd.observe(render, names="value")

# ---------------- Init ----------------
load_dataset(dataset_dd.value)
set_pressure_options()

display(widgets.VBox([widgets.HBox([dataset_dd, press_dd]), out]))
render()


**Inspecting nozzle behavior when it goes from Jetting -> [label] -> Jetting**

In [ ]:
# === Jet→label→Jet ONLY:
# === use ALL jetting points in each jetting phase (no K-window)
# === SINGLE CELL, NO CSVs: patterns are computed in-memory from labels matrix
# === Boxplot shows ONLY episode deltas (no Jet→Jet control)

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

plt.style.use("default")

# ---------- Config ----------
FIT_DIR   = "/content/drive/MyDrive/data/fit"
DATASETS  = ["1_lowering", "2_lowering", "1_increasing"]  # add more if needed

# ---------- Cache per dataset (avoid recompute on every tick) ----------
CACHE = {}  # dataset_key -> state dict

# ---------- Normalise labels ----------
def normalize_labels(labels_raw):
    labels_str = np.char.strip(labels_raw.astype(str))
    for bad in ("", "nan", "NaN", "NONE", "None", "none"):
        labels_str[labels_str == bad] = ""

    labels_low = np.char.lower(labels_str)

    # default bucket
    labels_cat = np.full(labels_low.shape, "other", dtype=object)

    labels_cat[labels_low == ""] = "unlabeled"
    labels_cat[labels_low == "jetting"] = "jetting"

    nonjet_variants = np.array(["non-jetting", "non jetting", "nonjetting", "non_jetting"])
    labels_cat[np.isin(labels_low, nonjet_variants)] = "nonjetting"
    labels_cat[labels_low == "deviated"] = "deviated"
    labels_cat[labels_low == "intermittent"] = "intermittent"
    labels_cat[labels_low == "other"] = "other"

    is_jet = (labels_cat == "jetting")
    # "bad" means any non-jet label of interest (unlabeled is NOT bad)
    is_bad = np.isin(labels_cat, ["deviated", "intermittent", "nonjetting", "other"])

    return labels_cat, is_jet, is_bad

# ---------- Helper: mean over a jetting phase ----------
def compute_phase_means(params6_all, noz_idx, idxs):
    """Return dict of mean A,tau,f,phi,C,D,Q over given pressure indices for a nozzle."""
    idxs = np.asarray(idxs, dtype=int)
    if idxs.size == 0:
        return None
    vals = params6_all[noz_idx, idxs, :]   # (#idxs, 6)
    means = vals.mean(axis=0)
    A, tau, f, phi, C, D = means
    Q = float(np.pi * f * tau)
    return {"A": float(A), "tau": float(tau), "f": float(f), "phi": float(phi), "C": float(C), "D": float(D), "Q": Q}

# ---------- Build patterns in-memory (no CSV) ----------
def find_jet_to_bad_to_jet_blocks(labels_cat_i):
    """
    Returns list of contiguous bad blocks (start,end) such that:
      - there is a contiguous jetting run immediately before the block
      - and a contiguous jetting run immediately after the block
    """
    P = labels_cat_i.shape[0]
    bad_mask = np.isin(labels_cat_i, ["deviated", "intermittent", "nonjetting", "other"])

    blocks = []
    j = 0
    while j < P:
        if not bad_mask[j]:
            j += 1
            continue

        start = j
        while j + 1 < P and bad_mask[j + 1]:
            j += 1
        end = j

        # contiguous jetting phase immediately before
        k = start - 1
        pre = []
        while k >= 0 and labels_cat_i[k] == "jetting":
            pre.append(k); k -= 1
        pre = pre[::-1]

        # contiguous jetting phase immediately after
        k = end + 1
        post = []
        while k < P and labels_cat_i[k] == "jetting":
            post.append(k); k += 1

        if len(pre) > 0 and len(post) > 0:
            blocks.append((start, end))

        j += 1

    return blocks

def compute_state_for_dataset(dataset_key: str):
    if dataset_key in CACHE:
        return CACHE[dataset_key]

    replicate = int(dataset_key.split("_")[0])
    variant   = dataset_key.split("_", 1)[1]

    MERGED_FP = os.path.join(
        FIT_DIR,
        f"merged_rep_{replicate}_{variant}_bankcorrected_params6_Q_rmse_nrmse_labels.npz"
    )

    if not os.path.exists(MERGED_FP):
        CACHE[dataset_key] = {"error": f"Missing merged NPZ with labels: {MERGED_FP}"}
        return CACHE[dataset_key]

    # ---------- Load merged NPZ ----------
    m = np.load(MERGED_FP, allow_pickle=True)
    params6_all = m["params6_all"]                # (N,P,6): [A, tau, f, phi, C, D]
    pressures   = m["pressures"].astype(float)    # (P,)
    nozzles     = m["nozzles"].astype(object)     # (N,)
    labels_raw  = m["labels"]                     # (N,P)

    N, P, _ = params6_all.shape

    # ---------- Normalize labels ----------
    labels_cat, is_jet, is_bad = normalize_labels(labels_raw)

    # ---------- Episode deltas ONLY: Jet phase before defect vs after defect ----------
    episode_rows = []

    for i in range(N):
        lbl_i = labels_cat[i]  # (P,)

        blocks = find_jet_to_bad_to_jet_blocks(lbl_i)
        if not blocks:
            continue

        for (start, end) in blocks:
            # jetting phase BEFORE block (contiguous jetting)
            pre_indices = []
            k = start - 1
            while k >= 0 and lbl_i[k] == "jetting":
                pre_indices.append(k); k -= 1
            pre_indices = pre_indices[::-1]

            # jetting phase AFTER block (contiguous jetting)
            post_indices = []
            k = end + 1
            while k < P and lbl_i[k] == "jetting":
                post_indices.append(k); k += 1

            if len(pre_indices) == 0 or len(post_indices) == 0:
                continue

            stats_before = compute_phase_means(params6_all, i, pre_indices)
            stats_after  = compute_phase_means(params6_all, i, post_indices)
            if stats_before is None or stats_after is None:
                continue

            defect_label_norm = str(lbl_i[start]).lower()  # deviated/intermittent/nonjetting/other

            rec = {
                "NozzleIdx": int(i),
                "NozzleID": str(nozzles[i]),
                "DefectLabelNorm": defect_label_norm,
                "DefectBlockStartIdx": int(start),
                "DefectBlockEndIdx": int(end),
                "DefectBlockStart_mBar": float(pressures[start]),
                "DefectBlockEnd_mBar": float(pressures[end]),
                "n_pre_phase": int(len(pre_indices)),
                "n_post_phase": int(len(post_indices)),
            }

            for name, val_before in stats_before.items():
                val_after = stats_after[name]
                rec[f"{name}_before"] = val_before
                rec[f"{name}_after"]  = val_after
                rec[f"d{name}"]       = float(val_after - val_before)

            episode_rows.append(rec)

    episode_df = pd.DataFrame(episode_rows)

    if len(episode_df) == 0:
        CACHE[dataset_key] = {"error": "No Jet→label→Jet episodes found – check label coverage."}
        return CACHE[dataset_key]

    delta_features = ["A", "tau", "f", "phi", "C", "D", "Q"]
    defect_labels_available = sorted(episode_df["DefectLabelNorm"].dropna().unique().tolist())

    CACHE[dataset_key] = dict(
        pressures=pressures,
        nozzles=nozzles,
        labels_cat=labels_cat,
        is_jet=is_jet,
        is_bad=is_bad,
        episode_df=episode_df,
        delta_features=delta_features,
        defect_labels_available=defect_labels_available,
        error=None,
    )
    return CACHE[dataset_key]

# ---------- Plot (episode-only; same boxplot representation) ----------
def plot_episode_deltas(state, dataset_key, feature, defect_label):
    col = f"d{feature}"
    df = state["episode_df"].dropna(subset=[col]).copy()

    if defect_label == "all":
        df_plot = df
        title_label_line = "All defect labels"
        print(f"Episode cases (Jet→label→Jet, all labels): {len(df_plot)}")
    else:
        df_plot = df[df["DefectLabelNorm"] == defect_label].copy()
        title_label_line = f"Defect label = '{defect_label}'"
        print(f"Episode cases (Jet→{defect_label}→Jet): {len(df_plot)}")

    if len(df_plot) == 0:
        print(f"\nNo episode data for Δ{feature} with the current defect-label filter.")
        return

    data   = [df_plot[col].values]
    labels = ["Jet→label→Jet"]

    plt.figure(figsize=(7, 5))
    bp = plt.boxplot(
        data,
        tick_labels=labels,
        patch_artist=True,
        showfliers=False,
    )

    # Keep same styling as your original
    for box in bp["boxes"]:
        box.set(facecolor="blue", alpha=0.35, edgecolor="blue", linewidth=1.5)
    for whisker in bp["whiskers"]:
        whisker.set(color="blue", linewidth=1.2)
    for cap in bp["caps"]:
        cap.set(color="blue", linewidth=1.2)
    for median in bp["medians"]:
        median.set(color="blue", linewidth=1.5)

    plt.ylabel(f"Δ{feature} = (post-jet phase) − (pre-jet phase)")
    plt.title(
        f"{dataset_key}\n"
        f"Δ{feature}: full jetting phases\n"
        f"Jet→label→Jet (episode only)\n"
        f"{title_label_line}"
    )
    plt.grid(axis="y", alpha=0.3)
    plt.tight_layout()
    plt.show()

# ---------- Widgets (dataset + feature + defect label) ----------
dataset_dd = widgets.Dropdown(options=DATASETS, value=DATASETS[0], description="Dataset")
feature_dd = widgets.Dropdown(options=["A","tau","f","phi","C","D","Q"], value="tau", description="Parameter")
defect_dd  = widgets.Dropdown(options=[("All","all")], value="all", description="Defect label")

out = widgets.Output()
UPDATING = False

def refresh_defect_options(ds_key):
    global UPDATING
    UPDATING = True

    st = compute_state_for_dataset(ds_key)
    if st.get("error"):
        defect_dd.options = [("All","all")]
        defect_dd.value = "all"
        UPDATING = False
        return

    labs = st["defect_labels_available"]
    defect_dd.options = [("All", "all")] + [(lab, lab) for lab in labs]
    if defect_dd.value not in [v for _, v in defect_dd.options]:
        defect_dd.value = "all"

    UPDATING = False

def render(_=None):
    if UPDATING:
        return
    ds_key = dataset_dd.value
    feat  = feature_dd.value
    dlab  = defect_dd.value

    st = compute_state_for_dataset(ds_key)

    with out:
        clear_output(wait=True)
        if st.get("error"):
            print(f"[{ds_key}] {st['error']}")
            return
        plot_episode_deltas(st, ds_key, feat, dlab)

def on_dataset_change(change):
    if change["name"] == "value":
        refresh_defect_options(change["new"])
        render()

dataset_dd.observe(on_dataset_change, names="value")
feature_dd.observe(render, names="value")
defect_dd.observe(render, names="value")

# init
refresh_defect_options(dataset_dd.value)
display(widgets.VBox([widgets.HBox([dataset_dd, feature_dd, defect_dd]), out]))
render()
